If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [1]:
! pip install datasets transformers

In [2]:
# training_type = 'base'
# training_type = 'distance'



training_type = 'base'
type_ = 'base_validations'

# training_type = 'distance'
# type_ = 'distance_validations'

# training_type = 'similar'
# type_ = 'similar_validations'

# training_type = 'cosine'
# type_ = 'cosine_validations'

# training_type = 'answer'
# type_ = 'answer_validations'

# training_type = 'entities'
# type_ = 'entities_validations'

# training_type = 'position'
# type_ = 'position_validations'

If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [3]:
# from huggingface_hub import notebook_login

# notebook_login()

Then you need to install Git-LFS. Uncomment the following instructions:

In [4]:
# !apt install git-lfs

Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [5]:
import transformers

print(transformers.__version__)

4.12.5


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/question-answering).

# Fine-tuning a model on a question-answering task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a question answering task, which is the task of extracting the answer to a question from a given context. We will see how to easily load a dataset for these kinds of tasks and use the `Trainer` API to fine-tune a model on it.

![Widget inference representing the QA task](https://github.com/huggingface/notebooks/blob/master/examples/images/question_answering.png?raw=1)

**Note:** This notebook finetunes models that answer question by taking a substring of a context, not by generating new text.

This notebook is built to run on any question answering task with the same format as SQUAD (version 1 or 2), with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a token classification head and a fast tokenizer (check on [this table](https://huggingface.co/transformers/index.html#bigtable) if this is the case). It might just need some small adjustments if you decide to use a different dataset than the one used here. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those three parameters, then the rest of the notebook should run smoothly:

In [6]:
# This flag is the difference between SQUAD v1 or 2 (if you're using another dataset, it indicates if impossible
# answers are allowed or not).
squad_v2 = False
# squad_v2 = True
# model_checkpoint = "distilbert-base-uncased"
model_checkpoint = "bert-base-uncased"
batch_size = 16

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [7]:
from datasets import load_dataset, load_metric

For our example here, we'll use the [SQUAD dataset](https://rajpurkar.github.io/SQuAD-explorer/). The notebook should work with any question answering dataset provided by the 🤗 Datasets library. If you're using your own dataset defined from a JSON or csv file (see the [Datasets documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) on how to load them), it might need some adjustments in the names of the columns used.

In [8]:
datasets = load_dataset("squad_v2" if squad_v2 else "squad")

Downloading:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/8.12M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

The `datasets` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set.

In [9]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

We can see the training, validation and test sets all have a column for the context, the question and the answers to those questions.

To access an actual element, you need to select a split first, then give an index:

### Dataset modification

In [10]:
import pandas as pd
import nltk

In [11]:
data = pd.DataFrame(datasets['train'])

## Data heuristics

#### Similar words heuristic split

In [12]:
from tqdm import tqdm

def count_similar_words_in_question_and_context(data):

    tokenizer = nltk.RegexpTokenizer(r"\w+")

    similar_words = []

    for i in tqdm(range(len(data))):
        context1 = nltk.word_tokenize(data['context'][i])
        question1 = nltk.word_tokenize(data['question'][i])
        context_new = [word for word in context1 if word.isalnum()]
        question_new = [word for word in question1 if word.isalnum()]
        similar_words.append(len(set(context_new).intersection(set(question_new))))
        # similar_words.append(len(set(data['context'][i].split()).intersection(set(data['question'][i].split()))))

    return similar_words

#### Answer length heuristic split

In [13]:
from nltk.tokenize import RegexpTokenizer

def average_answer_length(data):
    answers_text = []
    for i in range(len(data)):
        answers_text.append(data['answers'][i]['text'])
    answers_text

    answer_lenght = []
    tokenizer = RegexpTokenizer(r'\w+')
    for i in tqdm(range(len(data))):
        avg_lenght = 0
        for j in range(len(answers_text[i])):
            avg_lenght += len(tokenizer.tokenize(answers_text[i][j]))
        av = avg_lenght/(len(answers_text[i]))
        answer_lenght.append(av)

    return answer_lenght

#### Distance of answer from question word in context heuristic split

In [14]:
# code from this web site https://www.codegrepper.com/code-examples/python/find+index+of+sublist+in+list+python
def find_sub_list(sl,l):
    results=[]
    sll=len(sl)
    if sll <= 0:
        return results
#     print(f"Length is {len(sl)} and sl is {sl}")
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            results.append((ind,ind+sll-1))

    return results

In [15]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

def new_count_lowest_position_of_word_from_question_in_context(data):
    tokenizer = RegexpTokenizer(r'\w+')
    distances = []
    words = []

    for i in tqdm(range(len(data))):        
        indexes_of_words = []
        context_list = tokenizer.tokenize(data['context'][i])
        question_list = tokenizer.tokenize(data['question'][i])
        answer_text = tokenizer.tokenize(data['answers'][i]['text'][0])
        answer_start = data['answers'][i]['answer_start']

        indexes_of_words = find_sub_list(answer_text, context_list)

        if len(indexes_of_words) > 0:
            answer_index = indexes_of_words[0][0]
        else:
            distances.append(-1)
            words.append('None')
            continue

        filtered_words = [word for word in question_list if word not in stopwords.words('english')]

        list_indexes = {}

        for word in filtered_words:
            if word in context_list:
                for j in range(len(context_list)):
                    if word == context_list[j]:
                        list_indexes[abs(j - answer_index)] = context_list[j]

        sort_orders = sorted(list_indexes.items(), key=lambda x: x[0], reverse=False)

        if len(sort_orders) == 0:
            distances.append(-1)
            words.append('None')
        else:
            distances.append(sort_orders[0][0])
            words.append(sort_orders[0][1])

    return distances, words

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Cosine similarity from TF-IDF (context - question)

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm

vectorizer = TfidfVectorizer(stop_words='english', use_idf=True)
model = vectorizer.fit(data['context'])

def compute_similarity_between_context_and_question(data):
    similarities = []

    for i in tqdm(range(len(data))):
        context1 = vectorizer.transform([data['context'][i]])
        question1 = vectorizer.transform([data['question'][i]])
        similarities.append(cosine_similarity(context1, question1)[0][0])

    return similarities

#### Similar NERs to answer in context

In [17]:
import spacy
nlp = spacy.load('en_core_web_sm')


def show_ents(doc): 
    entities = []
    if doc.ents: 
        for ent in doc.ents: 
            entities.append(ent.label_)
    return entities


def count_similar_NER_from_context_to_answer(data):
    context_ents = []
    answer_ents = []

    for row in tqdm(range(len(data))):
        context = nlp(data['context'][row])
        context_ents.append(show_ents(context))
        for ans in data['answers'][row]['text']:
            act_ans_ents_ = []
            act_ans = nlp(ans)
            act_ans_ents_.append(show_ents(act_ans))
        answer_ents.append(act_ans_ents_)

    max_sim_ents = []

    for row, cont in tqdm(zip(answer_ents, context_ents)):
        max = 0
        for items in row:
            for item in items:
                if cont.count(item) > max:
                    max = cont.count(item)
        max_sim_ents.append(max)
    
    return max_sim_ents

#### Position of answer in context regarding question subject

In [18]:
def doc_pieces(doc):
    subjects = []
    for ent in doc:
        if ent.dep_ == 'nsubj':
            subjects.append(ent.text)
    return subjects

In [19]:
def find_subjects(data):
    q_subjects = []

    for item in tqdm(data['question']):
        question = nlp(item)
        q_subjects.append(doc_pieces(question))
    
    return q_subjects

In [20]:
import string
import re

def extract_answer_position_with_respect_to_subject(data, q_subjects):
#     q_subjects = []

#     for item in tqdm(data['question']):
#         question = nlp(item)
#         q_subjects.append(doc_pieces(question))

    positions = []

    for context, q_sub, answer in tqdm(zip(data['context'], q_subjects, data['answers'])):
        pos = 0
        max = 0
        for item in q_sub:
            if item in context:
                if '(' in item:
                    item = item.replace('(', '\(')
                indexes = [m.start() for m in re.finditer(item, context)]
                counter = 0
                for index in indexes:
                    if answer['answer_start'][0] < index:
                        break
                    else:
                        counter += 1
                if max < counter:
                    max = counter
                pos = max
            else:
                pos = -1
        positions.append(pos)
    
    return positions

#### Compute heuristics for data

In [21]:
# data['similar_words'] = count_similar_words_in_question_and_context(data)

In [22]:
# data['answer_lenght'] = average_answer_length(data)

In [23]:
# data['distances'], closest_words = new_count_lowest_position_of_word_from_question_in_context(data)

In [24]:
# data['cosine_similarity'] = compute_similarity_between_context_and_question(data)

In [25]:
# data['max_sim_ents'] = count_similar_NER_from_context_to_answer(data)

In [26]:
# question_subjects = find_subjects(data)

In [27]:
# data['answer_subject_positions'] = extract_answer_position_with_respect_to_subject(data, question_subjects)

In [28]:
# threshold = 7
# data['dist_flag'] = [1 if x > threshold else 0 for x in data['distances']]

# threshold = 4
# data['sim_flag'] = [1 if x <= threshold else 0 for x in data['similar_words']]

# threshold = 3
# data['ans_flag'] = [1 if x > threshold else 0 for x in data['answer_lenght']]

# threshold = 0.1
# data['cos_flag'] = [1 if x <= threshold else 0 for x in data['cosine_similarity']]

# threshold = 1
# data['pos_flag'] = [1 if x > threshold else 0 for x in data['answer_subject_positions']]

# threshold = 0
# data['ents_flag'] = [1 if x <= threshold else 0 for x in data['max_sim_ents']]

In [29]:
# data.to_json('squad_train_with_heuristics_flags.json', orient='records')

In [30]:
data = pd.read_json('../input/squad-train-supersampled-all-heuristics/squad_train_with_heuristics_flags.json')

In [31]:
# data['hardness'] = data[['dist_flag','sim_flag','ans_flag', 'cos_flag', 'pos_flag', 'ents_flag']].sum(axis=1)

In [32]:
# data['hardness'].value_counts()

In [33]:
# data['similar_words'] = count_similar_words_in_question_and_context(data)
# data

In [34]:
# data_higher, data_lower = [x for _, x in data.groupby(data['similar_words'] <= 4)]

In [35]:
# data_higher, data_lower = [x for _, x in data.groupby(data['answer_lenght'] <= 3)]

In [36]:
# data_higher, data_lower = [x for _, x in data.groupby(data['cosine_similarity'] <= 0.1)]

In [37]:
# data_higher, data_lower = [x for _, x in data.groupby(data['answer_subject_positions'] <= 1)]
# answer_subject_positions

In [38]:
# data_higher, data_lower = [x for _, x in data.groupby(data['max_sim_ents'] <= 0)]

In [39]:
# data_higher, data_lower = [x for _, x in data.groupby(data['hardness'] <= 2)]

In [40]:
# data['answer_lenght'] = average_answer_length(data)
# data

In [41]:
# data['distances'], closest_words = new_count_lowest_position_of_word_from_question_in_context(data)

In [42]:
# data_higher, data_lower = [x for _, x in data.groupby(data['distances'] <= 7)]

In [43]:
# len(data_higher)

In [44]:
# data_higher_length = len(data_higher)

In [45]:
# len(data_lower)

In [46]:
# if len(data_higher) > len(data_lower):
#     new_data = data_lower.append(data_higher.sample(len(data_lower)), ignore_index=True)
# else:
#     new_data = data_higher.append(data_lower.sample(len(data_higher)), ignore_index=True)

In [47]:
def supersample_dataset(data_lower, data_higher):
    list_subsets = []
    list_subsets.append(data_lower)
    list_subsets.append(data_higher)

    if len(data_higher) > len(data_lower):
        for i in range(len(data_higher)//len(data_lower)):
#             print(f"higher {i}")
            list_subsets.append(data_lower.sample(frac=1))
        new_data = pd.concat(list_subsets)
    else:
        for i in range(len(data_lower)//len(data_higher)):
#             print(f"lower {i}")
            list_subsets.append(data_higher.sample(frac=1))
        new_data = pd.concat(list_subsets)
    
    return new_data

In [48]:
! python --version

Python 3.7.12


In [49]:
if training_type == 'base':
    new_data = data
elif training_type == 'distance':
    data_higher, data_lower = [x for _, x in data.groupby(data['distances'] <= 7)]
    new_data = supersample_dataset(data_lower, data_higher)
elif training_type == 'similar':
    data_higher, data_lower = [x for _, x in data.groupby(data['similar_words'] <= 4)]
    new_data = supersample_dataset(data_lower, data_higher)
elif training_type == 'cosine':
    data_higher, data_lower = [x for _, x in data.groupby(data['cosine_similarity'] <= 0.1)]
    new_data = supersample_dataset(data_lower, data_higher)
elif training_type == 'answer':
    data_higher, data_lower = [x for _, x in data.groupby(data['answer_lenght'] <= 3)]
    new_data = supersample_dataset(data_lower, data_higher)
elif training_type == 'entities':
    data_higher, data_lower = [x for _, x in data.groupby(data['max_sim_ents'] <= 0)]
    new_data = supersample_dataset(data_lower, data_higher)
elif training_type == 'position':
    data_higher, data_lower = [x for _, x in data.groupby(data['answer_subject_positions'] <= 1)]
    new_data = supersample_dataset(data_lower, data_higher)
elif training_type == 'all':
    new_data = pd.concat([data.query('dist_flag == 0 and sim_flag == 0 and ans_flag == 0 and cos_flag == 0 and pos_flag == 0 and ents_flag == 0'),
                        data.query('dist_flag == 1'),
                        data.query('sim_flag == 1'),
                        data.query('ans_flag == 1'),
                        data.query('cos_flag == 1'),
                        data.query('pos_flag == 1'),
                        data.query('ents_flag == 1')]).sample(frac=1)
elif training_type == 'top_3':
    new_data = pd.concat([data,
                    data.query('ans_flag == 1'),
                    data.query('cos_flag == 1'),
                    data.query('cos_flag == 1'),
                    data.query('cos_flag == 1'),
                    data.query('dist_flag == 1'),
                    data.query('dist_flag == 1')]).sample(frac=1)
elif training_type == 'dist_cos':
    new_data = pd.concat([data,
                    data.query('cos_flag == 1'),
                    data.query('cos_flag == 1'),
                    data.query('cos_flag == 1'),
                    data.query('cos_flag == 1'),
                    data.query('dist_flag == 1'),
                    data.query('dist_flag == 1'),
                    data.query('dist_flag == 1')]).sample(frac=1)
elif training_type == 'dist_ans':
    new_data = pd.concat([data,
                    data.query('ans_flag == 1'),
                    data.query('dist_flag == 1'),
                    data.query('dist_flag == 1'),
                    data.query('dist_flag == 1')]).sample(frac=1)
elif training_type == 'ans_cos':
    new_data = pd.concat([data,
                    data.query('ans_flag == 1'),
                    data.query('cos_flag == 1'),
                    data.query('cos_flag == 1'),
                    data.query('cos_flag == 1'),
                    data.query('cos_flag == 1')]).sample(frac=1)
else:
    print('something went wrong! wrong type.')

In [50]:
# list_subsets = []
# list_subsets.append(data_lower)
# list_subsets.append(data_higher)

# # if len(data_higher) > len(data_lower):
# #     for i in range(20):
# #         list_subsets.append(data_lower.sample((len(data_higher) - len(data_lower)) // 20,  ignore_index=True))
# #     new_data = pd.concat(list_subsets)
# # else:
# #     for i in range(20):
# #         list_subsets.append(data_higher.sample((len(data_lower) - len(data_higher)) // 20,  ignore_index=True))
# #     new_data = pd.concat(list_subsets)
    
# if len(data_higher) > len(data_lower):
#     for i in range(len(data_higher)//len(data_lower)):
#         print(f"higher {i}")
#         list_subsets.append(data_lower.sample(frac=1))
#     new_data = pd.concat(list_subsets)
# else:
#     for i in range(len(data_lower)//len(data_higher)):
#         print(f"lower {i}")
#         list_subsets.append(data_higher.sample(frac=1))
#     new_data = pd.concat(list_subsets)

In [51]:
# new_data = new_data.sample(frac=1)

In [52]:
# new_data = new_data.drop('similar_words', axis=1)

In [53]:
# new_data = new_data.drop('answer_lenght', axis=1)

In [54]:
# new_data = new_data.drop('distances', axis=1)

In [55]:
# new_data = pd.concat([data.query('dist_flag == 0 and sim_flag == 0 and ans_flag == 0 and cos_flag == 0 and pos_flag == 0 and ents_flag == 0'),
#                     data.query('dist_flag == 1'),
#                     data.query('sim_flag == 1'),
#                     data.query('ans_flag == 1'),
#                     data.query('cos_flag == 1'),
#                     data.query('pos_flag == 1'),
#                     data.query('ents_flag == 1')]).sample(frac=1)

In [56]:
# new_data = pd.concat([data.query('dist_flag == 1'),
#                     data.query('sim_flag == 1'),
#                     data.query('ans_flag == 1'),
#                     data.query('cos_flag == 1'),
#                     data.query('dist_flag == 1'),
#                     data.query('pos_flag == 1'),
#                     data.query('ents_flag == 1')]).sample(frac=1)

In [57]:
# data_dist = data.query('dist_flag == 1')
# data_dist

In [58]:
# data_ans = data.query('ans_flag == 1')
# data_ans

In [59]:
# data_cos = data.query('cos_flag == 1')
# data_cos

In [60]:
# new_data = pd.concat([data,
#                     data.query('ans_flag == 1'),
#                     data.query('dist_flag == 1'),
#                     data.query('dist_flag == 1'),
#                     data.query('dist_flag == 1')]).sample(frac=1)

In [61]:
# new_data

In [62]:
# # new_data = new_data.drop(['distances', 'similar_words', 'answer_lenght', 'cosine_similarity', 'answer_subject_positions', 'max_sim_ents', 'dist_flag', 'sim_flag', 'ans_flag', 'cos_flag', 'pos_flag', 'ents_flag', 'hardness'], axis = 1)
# new_data = new_data.drop(['distances', 'similar_words', 'answer_lenght', 'cosine_similarity', 'answer_subject_positions', 'max_sim_ents', 'dist_flag', 'sim_flag', 'ans_flag', 'cos_flag', 'pos_flag', 'ents_flag'], axis = 1)

In [63]:
# new_data

In [64]:
# len(new_data)

In [65]:
from datasets import Dataset

In [66]:
nq_val_dataset = Dataset.from_pandas(pd.read_json('../input/nq-dev-formatted/nq_dev_formated.json'))

In [67]:
adversarialqa_val_dataset = Dataset.from_pandas(pd.read_json('../input/adversarialqa-valid/adversarialQA_validation.json'))

In [68]:
triviaqa_val_dataset = Dataset.from_pandas(pd.read_json('../input/triviaqa-dev-formatted/triviaqa_dev_formated.json'))

In [69]:
if training_type != 'base':
    new_data = new_data.drop(['distances', 'similar_words', 'answer_lenght', 'cosine_similarity', 'answer_subject_positions', 'max_sim_ents', 'dist_flag', 'sim_flag', 'ans_flag', 'cos_flag', 'pos_flag', 'ents_flag'], axis = 1)
    datasets['train'] = Dataset.from_pandas(new_data)
    datasets['train'] = datasets['train'].remove_columns("__index_level_0__")

In [70]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [71]:
datasets["train"][0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

We can see the answers are indicated by their start position in the text (here at character 515) and their full text, which is a substring of the context as we mentioned above.

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset (automatically decoding the labels in passing).

In [72]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [73]:
show_random_elements(datasets["train"])

,id,title,context,question,answers
0,572939f06aef051400154b94,Insect,"The abdomen is the largest tagma of the insect, which typically consists of 11–12 segments and is less strongly sclerotized than the head or thorax. Each segment of the abdomen is represented by a sclerotized tergum and sternum. Terga are separated from each other and from the adjacent sterna or pleura by membranes. Spiracles are located in the pleural area. Variation of this ground plan includes the fusion of terga or terga and sterna to form continuous dorsal or ventral shields or a conical tube. Some insects bear a sclerite in the pleural area called a laterotergite. Ventral sclerites are sometimes called laterosternites. During the embryonic stage of many insects and the postembryonic stage of primitive insects, 11 abdominal segments are present. In modern insects there is a tendency toward reduction in the number of the abdominal segments, but the primitive number of 11 is maintained during embryogenesis. Variation in abdominal segment number is considerable. If the Apterygota are considered to be indicative of the ground plan for pterygotes, confusion reigns: adult Protura have 12 segments, Collembola have 6. The orthopteran family Acrididae has 11 segments, and a fossil specimen of Zoraptera has a 10-segmented abdomen.",What is the largest tagma of an insect?,"{'text': ['abdomen'], 'answer_start': [4]}"
1,5727de793acd2414000dee87,Gramophone_record,"New or ""virgin"" heavy/heavyweight (180–220 g) vinyl is commonly used for modern audiophile vinyl releases in all genres. Many collectors prefer to have heavyweight vinyl albums, which have been reported to have better sound than normal vinyl because of their higher tolerance against deformation caused by normal play. 180 g vinyl is more expensive to produce only because it uses more vinyl. Manufacturing processes are identical regardless of weight. In fact, pressing lightweight records requires more care. An exception is the propensity of 200 g pressings to be slightly more prone to non-fill, when the vinyl biscuit does not sufficiently fill a deep groove during pressing (percussion or vocal amplitude changes are the usual locations of these artifacts). This flaw causes a grinding or scratching sound at the non-fill point.","Which pressing of vinyl require more attention to detail, lightweight or heavyweight?","{'text': ['lightweight records requires more care'], 'answer_start': [471]}"
2,5727d9374b864d1900163e7a,Sahara,"To the south, the Sahara is bounded by the Sahel, a belt of dry tropical savanna with a summer rainy season that extends across Africa from east to west. The southern limit of the Sahara is indicated botanically by the southern limit of Cornulaca monacantha (a drought-tolerant member of the Chenopodiaceae), or northern limit of Cenchrus biflorus, a grass typical of the Sahel. According to climatic criteria, the southern limit of the Sahara corresponds to the 150 mm (5.9 in) isohyet of annual precipitation (this is a long-term average, since precipitation varies annually).",What is the long term precipitation average of the Sahara?,"{'text': ['150 mm'], 'answer_start': [463]}"
3,56cd7dda62d2951400fa6640,Sino-Tibetan_relations_during_the_Ming_dynasty,"Marsha Weidner states that Deshin Shekpa's miracles ""testified to the power of both the emperor and his guru and served as a legitimizing tool for the emperor's problematic succession to the throne,"" referring to the Yongle Emperor's conflict with the previous Jianwen Emperor. Tsai writes that Deshin Shekpa aided the legitimacy of the Yongle Emperor's rule by providing him with portents and omens which demonstrated Heaven's favor of the Yongle Emperor on the Ming throne.",Who did the Yongle Emperor have a conflict with?,"{'text': ['Jianwen Emperor'], 'answer_start': [261]}"
4,573217e8e17f3d140042267f,Pacific_War,"U.S. submarines, as well as some British and Dutch vessels, operating from bases at Cavite in the Philippines (1941–42); Fremantle

## Preprocessing the training data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [74]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

The following assertion ensures that our tokenizer is a fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, and we will need some of the special features they have for our preprocessing.

In [75]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

You can check which type of models have a fast tokenizer available and which don't on the [big table of models](https://huggingface.co/transformers/index.html#bigtable).

You can directly call this tokenizer on two sentences (one for the answer, one for the context):

In [76]:
tokenizer("What is your name?", "My name is Sylvain.")

{'input_ids': [101, 2054, 2003, 2115, 2171, 1029, 102, 2026, 2171, 2003, 25353, 22144, 2378, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

Now one specific thing for the preprocessing in question answering is how to deal with very long documents. We usually truncate them in other tasks, when they are longer than the model maximum sentence length, but here, removing part of the the context might result in losing the answer we are looking for. To deal with this, we will allow one (long) example in our dataset to give several input features, each of length shorter than the maximum length of the model (or the one we set as a hyper-parameter). Also, just in case the answer lies at the point we split a long context, we allow some overlap between the features we generate controlled by the hyper-parameter `doc_stride`:

In [77]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

Let's find one long example in our dataset:

In [78]:
for i, example in enumerate(datasets["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        break
example = datasets["train"][i]

Without any truncation, we get the following length for the input IDs:

In [79]:
len(tokenizer(example["question"], example["context"])["input_ids"])

396

Now, if we just truncate, we will lose information (and possibly the answer to our question):

In [80]:
len(tokenizer(example["question"], example["context"], max_length=max_length, truncation="only_second")["input_ids"])

384

Note that we never want to truncate the question, only the context, else the `only_second` truncation picked. Now, our tokenizer can automatically return us a list of features capped by a certain maximum length, with the overlap we talked above, we just have to tell it with `return_overflowing_tokens=True` and by passing the stride:

In [81]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

Now we don't have one list of `input_ids`, but several: 

In [82]:
[len(x) for x in tokenized_example["input_ids"]]

[384, 157]

And if we decode them, we can see the overlap:

In [83]:
for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))

[CLS] how many wins does the notre dame men's basketball team have? [SEP] the men's basketball team has over 1, 600 wins, one of only 12 schools who have reached that mark, and have appeared in 28 ncaa tournaments. former player austin carr holds the record for most points scored in a single game of the tournament with 61. although the team has never won the ncaa tournament, they were named by the helms athletic foundation as national champions twice. the team has orchestrated a number of upsets of number one ranked teams, the most notable of which was ending ucla's record 88 - game winning streak in 1974. the team has beaten an additional eight number - one teams, and those nine wins rank second, to ucla's 10, all - time in wins against the top team. the team plays in newly renovated purcell pavilion ( within the edmund p. joyce center ), which reopened for the beginning of the 2009 – 2010 season. the team is coached by mike brey, who, as of the 2014 – 15 season, his fifteenth at notr

Now this will give us some work to properly treat the answers: we need to find in which of those features the answer actually is, and where exactly in that feature. The models we will use require the start and end positions of these answers in the tokens, so we will also need to to map parts of the original context to some tokens. Thankfully, the tokenizer we're using can help us with that by returning an `offset_mapping`:

In [84]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 3), (4, 8), (9, 13), (14, 18), (19, 22), (23, 28), (29, 33), (34, 37), (37, 38), (38, 39), (40, 50), (51, 55), (56, 60), (60, 61), (0, 0), (0, 3), (4, 7), (7, 8), (8, 9), (10, 20), (21, 25), (26, 29), (30, 34), (35, 36), (36, 37), (37, 40), (41, 45), (45, 46), (47, 50), (51, 53), (54, 58), (59, 61), (62, 69), (70, 73), (74, 78), (79, 86), (87, 91), (92, 96), (96, 97), (98, 101), (102, 106), (107, 115), (116, 118), (119, 121), (122, 126), (127, 138), (138, 139), (140, 146), (147, 153), (154, 160), (161, 165), (166, 171), (172, 175), (176, 182), (183, 186), (187, 191), (192, 198), (199, 205), (206, 208), (209, 210), (211, 217), (218, 222), (223, 225), (226, 229), (230, 240), (241, 245), (246, 248), (248, 249), (250, 258), (259, 262), (263, 267), (268, 271), (272, 277), (278, 281), (282, 285), (286, 290), (291, 301), (301, 302), (303, 307), (308, 312), (313, 318), (319, 321), (322, 325), (326, 330), (330, 331), (332, 340), (341, 351), (352, 354), (355, 363), (364, 373), (374,

This gives, for each index of our input IDS, the corresponding start and end character in the original text that gave our token. The very first token (`[CLS]`) has (0, 0) because it doesn't correspond to any part of the question/answer, then the second token is the same as the characters 0 to 3 of the question:

In [85]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

how How


So we can use this mapping to find the position of the start and end tokens of our answer in a given feature. We just have to distinguish which parts of the offsets correspond to the question and which part correspond to the context, this is where the `sequence_ids` method of our `tokenized_example` can be useful:

In [86]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

It returns `None` for the special tokens, then 0 or 1 depending on whether the corresponding token comes from the first sentence past (the question) or the second (the context). Now with all of this, we can find the first and last token of the answer in one of our input feature (or if the answer is not in this feature):

In [87]:
answers = example["answers"]
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])

# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")

23 26


And we can double check that it is indeed the theoretical answer:

In [88]:
print(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
print(answers["text"][0])

over 1, 600
over 1,600


For this notebook to work with any kind of models, we need to account for the special case where the model expects padding on the left (in which case we switch the order of the question and the context):

In [89]:
pad_on_right = tokenizer.padding_side == "right"

Now let's put everything together in one function we will apply to our training set. In the case of impossible answers (the answer is in another feature given by an example with a long context), we set the cls index for both the start and end position. We could also simply discard those examples from the training set if the flag `allow_impossible_answers` is `False`. Since the preprocessing is already complex enough as it is, we've kept is simple for this part.

In [90]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [91]:
features = prepare_train_features(datasets['train'][:5])

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command. Since our preprocessing changes the number of samples, we need to remove the old columns when applying it.

In [92]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

  0%|          | 0/88 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready for training, we can download the pretrained model and fine-tune it. Since our task is question answering, we use the `AutoModelForQuestionAnswering` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us:

In [93]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [94]:
from transformers import EarlyStoppingCallback

model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-squad_with_callbacks",
#     evaluation_strategy = "epoch",
    evaluation_strategy = "steps",
    eval_steps = 200, # Evaluation and Save happens every 10 steps
    save_steps = 200,
    logging_steps = 200,
    save_total_limit = 5, # Only last 5 models are saved. Older ones are deleted.
    learning_rate=2e-5,
#     warmup_ratio=0.1,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to="none",
    load_best_model_at_end=True,
#     metric_for_best_model = 'f1',
#     push_to_hub=False,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay.

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/bert-finetuned-squad"` or `"huggingface/bert-finetuned-squad"`).

Then we will need a data collator that will batch our processed examples together, here the default one will work:

In [95]:
from transformers import default_data_collator

data_collator = default_data_collator

We will evaluate our model and compute metrics in the next section (this is a very long operation, so we will only compute the evaluation loss during training).

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [96]:
tokenized_nq_val_dataset = nq_val_dataset.map(prepare_train_features, batched=True, remove_columns=nq_val_dataset.column_names)

  0%|          | 0/3 [00:00<?, ?ba/s]

In [97]:
tokenized_adversarialqa_val_dataset = adversarialqa_val_dataset.map(prepare_train_features, batched=True, remove_columns=adversarialqa_val_dataset.column_names)

  0%|          | 0/3 [00:00<?, ?ba/s]

In [98]:
tokenized_triviaqa_val_dataset = triviaqa_val_dataset.map(prepare_train_features, batched=True, remove_columns=triviaqa_val_dataset.column_names)

  0%|          | 0/10 [00:00<?, ?ba/s]

In [99]:
from transformers import TrainerCallback, TrainerState, TrainerControl

class MyCallback(TrainerCallback):
    "A callback that evaluates model on other datasets"
    
    def __init__(self, trainer, dataset_nq, dataset_adv, dataset_trivia): #, metric_key_prefix, num_beams):
        super().__init__()
        self.trainer = trainer
        self.dataset_nq = dataset_nq
        self.dataset_adv = dataset_adv
        self.dataset_trivia = dataset_trivia
        
#         self.metric_key_prefix = metric_key_prefix
#         self.num_beams = num_beams
    
    def on_save(self, args: TrainingArguments, state: TrainerState, control: TrainerControl, **kwargs):
#         print("Starting training")
        eval_ = self.trainer.evaluate(self.dataset_nq)
        print(f"Natural questions: {eval_['eval_loss']}")
        
        eval_ = self.trainer.evaluate(self.dataset_adv)
        print(f"AdversarialQA: {eval_['eval_loss']}")
        
        eval_ = self.trainer.evaluate(self.dataset_trivia)
        print(f"TriviaQA: {eval_['eval_loss']}")
#         return eval_['eval_loss']

In [100]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,    
    callbacks = [EarlyStoppingCallback(early_stopping_patience=120)]
)

In [101]:
trainer.add_callback(MyCallback(trainer, tokenized_nq_val_dataset, tokenized_adversarialqa_val_dataset, tokenized_triviaqa_val_dataset))

We can now finetune our model by just calling the `train` method:

In [102]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [103]:
trainer.train()

***** Running training *****
  Num examples = 88524
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 16599


Step,Training Loss,Validation Loss
200,3.294300,2.131329
200,3.294300,3.062943
200,3.294300,4.345679
200,3.294300,3.782588
400,1.988100,1.681308
400,1.988100,2.374461
400,1.988100,4.008904
400,1.988100,4.241457
600,1.668800,1.490974
600,1.668800,2.108654


***** Running Evaluation *****
  Num examples = 10784
  Batch size = 16
Saving model checkpoint to bert-base-uncased-finetuned-squad_with_callbacks/checkpoint-200
Configuration saved in bert-base-uncased-finetuned-squad_with_callbacks/checkpoint-200/config.json
Model weights saved in bert-base-uncased-finetuned-squad_with_callbacks/checkpoint-200/pytorch_model.bin
tokenizer config file saved in bert-base-uncased-finetuned-squad_with_callbacks/checkpoint-200/tokenizer_config.json
Special tokens file saved in bert-base-uncased-finetuned-squad_with_callbacks/checkpoint-200/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 2388
  Batch size = 16
***** Running Evaluation *****
  Num examples = 3010
  Batch size = 16


Natural questions: 3.062943458557129


***** Running Evaluation *****
  Num examples = 36286
  Batch size = 16


...

***** Running Evaluation *****
  Num examples = 10784
  Batch size = 16
Saving model checkpoint to bert-base-uncased-finetuned-squad_with_callbacks/checkpoint-10200
Configuration saved in bert-base-uncased-finetuned-squad_with_callbacks/checkpoint-10200/config.json
Model weights saved in bert-base-uncased-finetuned-squad_with_callbacks/checkpoint-10200/pytorch_model.bin
tokenizer config file saved in bert-base-uncased-finetuned-squad_with_callbacks/checkpoint-10200/tokenizer_config.json
Special tokens file saved in bert-base-uncased-finetuned-squad_with_callbacks/checkpoint-10200/special_tokens_map.json
Deleting older checkpoint [bert-base-uncased-finetuned-squad_with_callbacks/checkpoint-9200] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 2388
  Batch size = 16
***** Running Evaluation *****
  Num examples = 3010
  Batch size = 16


Natural questions: 1.936430811882019


***** Running Evaluation *****
  Num examples = 36286
  Batch size = 16


AdversarialQA: 3.388887405395508


KeyboardInterrupt: 

Since this training is particularly long, let's save the model just in case we need to restart.

In [ ]:
trainer.save_model("test-squad-trained")

In [ ]:
import shutil
# shutil.make_archive(f"{model_name}-finetuned-squad_with_callbacks_reduced_on_answer_length_3", 'zip', './test-squad-trained')
# shutil.make_archive(f"{model_name}-finetuned-squad_with_callbacks_reduced_on_similar_words_4_super", 'zip', './test-squad-trained')

# shutil.make_archive(f"{model_name}-finetuned-squad_with_callbacks_reduced_on_answer_length_3_super", 'zip', './test-squad-trained')
# shutil.make_archive(f"{model_name}-finetuned-squad_with_callbacks_all_super_dd_ns", 'zip', './test-squad-trained')

# shutil.make_archive(f"{model_name}-finetuned-squad_with_callbacks_reduced_on_cosine_similarity_01_super", 'zip', './test-squad-trained')
# shutil.make_archive(f"{model_name}-finetuned-squad_with_callbacks_reduced_on_ans_sub_pos_1_super", 'zip', './test-squad-trained')
# shutil.make_archive(f"{model_name}-finetuned-squad_with_callbacks_reduced_on_sim_ents_0_super", 'zip', './test-squad-trained')
# shutil.make_archive(f"{model_name}-finetuned-squad_with_callbacks_super_on_all_2", 'zip', './test-squad-trained')
# shutil.make_archive(f"{model_name}-finetuned-squad_with_callbacks_super_top_3", 'zip', './test-squad-trained')
# shutil.make_archive(f"{model_name}-finetuned-squad_with_callbacks_super_dist_cos", 'zip', './test-squad-trained')

# shutil.make_archive(f"{model_name}-finetuned-squad_with_callbacks_super_dist_ans", 'zip', './test-squad-trained')


# shutil.make_archive(f"{model_name}-finetuned-squad_with_callbacks_base_v2", 'zip', './test-squad-trained')

# shutil.make_archive(f"{model_name}-finetuned-squad_with_callbacks_distances_7_v2", 'zip', './test-squad-trained')


# shutil.make_archive(f"{model_name}-finetuned-squad_with_callbacks_base_30pat", 'zip', './test-squad-trained')

shutil.make_archive(f"{model_name}-finetuned-squad_with_callbacks_{type_}", 'zip', './test-squad-trained')